In [1]:
from alpha_vantage.cryptocurrencies import CryptoCurrencies
import pandas_ta as ta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import os

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

key = os.environ['av_key']

In [2]:
cc = CryptoCurrencies(key=key, output_format='pandas')

In [3]:
df, meta_data = cc.get_crypto_intraday('ETH', market='USD', interval='1min', outputsize='full')

In [4]:
df = df.rename(columns={"4. close": "close",
                   "1. open": "open",
                   "2. high": "high",
                   "3. low": "low",
                   "5. volume": "volume"})

df = df[['open', 'high', 'low', 'close', 'volume']]

In [5]:
df = df.sort_index()
df['idx_int'] = np.arange(0, len(df))
df = df.reset_index()

In [6]:
df.head()

,date,open,high,low,close,volume,idx_int
0,2024-11-06 12:19:00,2648.29,2648.99,2647.08,2647.79,94.0,0
1,2024-11-06 12:20:00,2647.79,2648.21,2641.00,2641.01,124.0,1
2,2024-11-06 12:21:00,2641.00,2645.33,2640.55,2643.85,105.0,2
3,2024-11-06 12:22:00,2643.86,2647.71,2643.86,2647.64,67.0,3
4,2024-11-06 12:23:00,2647.60,2647.60,2644.64,2645.34,398.0,4


In [7]:
def compute_gradient(start_index, df, x_label, y_label, len_data):
    # Ensure we only take data points from n to n+5
    if start_index + 5 > len(df):
        return None  # Return None if there are not enough points to calculate gradient
    
    # Extract the last 5 data points (x, y) from the DataFrame
    data_segment = df.iloc[start_index:start_index + len_data]
    x = data_segment[x_label]
    y = data_segment[y_label]
    
    # Calculate the necessary summations for the least squares formula
    n = len(x)
    sum_x = x.sum()
    sum_y = y.sum()
    sum_x2 = (x**2).sum()
    sum_xy = (x * y).sum()
    
    # Calculate the slope (gradient) using the least squares formula
    slope = (n * sum_xy - sum_x * sum_y) / (n * sum_x2 - sum_x**2)
    return slope

In [8]:
df['gradient_ls'] = np.nan  # Initialize the column with None
len_data = 5

# Loop through the DataFrame, compute gradient for each row (starting point)
for i in range(len(df) - 5):  # Make sure we have at least 5 points for each calculation
    gradient = compute_gradient(i, df, 'idx_int', 'close', len_data)
    df.at[i + len_data, 'gradient_ls'] = gradient  # Store the gradient in the row corresponding to n+4

In [9]:
# realtime stock trading
df.ta.ema(length=9, append=True)
df.ta.ema(length=21, append=True)
df.ta.rsi(length=7, append=True)
# df.ta.rsi(length=12, append=True)
# df.ta.rsi(length=24, append=True)
df.ta.bbands(length=20, std=2, append=True)
df.ta.macd(fast=6, slow=13, signal=5, append=True)
df.ta.psar(append=True)

,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2
0,NaN,NaN,0.02,0
1,NaN,2666.640000,0.04,0
2,NaN,2665.614400,0.06,0
3,NaN,2664.110536,0.06,0
4,NaN,2662.696904,0.06,0
...,...,...,...,...
345,2652.790364,NaN,0.06,0
346,2653.494142,NaN,0.08,0
347,2654.411411,NaN,0.10,0
348,2655.496270,NaN,0.12,0


In [10]:
df['r_ema_s_m'] = df['EMA_9'] / df['EMA_21']

In [11]:
df['gradient_ema9'] = np.nan  # Initialize the column with None
len_data = 5

# Loop through the DataFrame, compute gradient for each row (starting point)
for i in range(len(df) - 3):  # Make sure we have at least 5 points for each calculation
    gradient = compute_gradient(i, df, 'idx_int', 'EMA_9', len_data)
    df.at[i + len_data, 'gradient_ema_9'] = gradient  # Store the gradient in the row corresponding to n+4
    gradient = compute_gradient(i, df, 'idx_int', 'RSI_7', len_data)
    df.at[i + len_data, 'gradient_rsi_7'] = gradient  # Store the gradient in the row corresponding to n+4

In [12]:
mask_crossing = (df['r_ema_s_m']<1.0005)&(df['r_ema_s_m']>0.9995)
df['flag_ema_crossing'] = 0
df.loc[mask_crossing, 'flag_ema_crossing'] = 1

In [13]:
mask_ema_grad_pos = (df['gradient_ema_9']>0.05)
mask_ema_grad_neg = (df['gradient_ema_9']<-0.05)
df['flag_grad_ema'] = 0
df.loc[mask_ema_grad_pos, 'flag_grad_ema'] = 1
df.loc[mask_ema_grad_neg, 'flag_grad_ema'] = -1

mask_rsi_grad_pos = (df['gradient_rsi_7']>=1)
mask_rsi_grad_neg = (df['gradient_rsi_7']<=1)
df['flag_grad_rsi'] = 0
df.loc[mask_rsi_grad_pos, 'flag_grad_rsi'] = 1
df.loc[mask_rsi_grad_neg, 'flag_grad_rsi'] = -1

In [14]:
# df.ta.rsi(length=6, append=True)
# df.ta.rsi(length=12, append=True)
# df.ta.rsi(length=24, append=True)
# df.ta.bbands(length=20, std=2, append=True)
# df.ta.macd(fast=15, slow=30, signal=9, append=True)
# df.ta.psar(append=True)

In [16]:
df['ema_short_above_or_below'] = 0
df.loc[(df['EMA_9']>df['EMA_21']), 'ema_short_above_or_below'] = 1
df.loc[(df['EMA_9']<df['EMA_21']), 'ema_short_above_or_below'] = -1

In [31]:
df['long_entry'] = np.nan
df['short_entry'] = np.nan
df['long_exit'] = np.nan
df['short_exit'] = np.nan

# Strategy

With access to real-time crypto data on a 1-minute frequency, you can use a high-frequency trading strategy that relies on these indicators to capture quick, small movements. Here’s a 1-minute crypto trading strategy using Bollinger Bands, MACD, RSI, Parabolic SAR, and EMA.

### Strategy Overview

In this strategy:
- **Indicators**:
  - **Bollinger Bands (BB)**: Used for spotting volatility squeezes and reversal points.
  - **MACD**: Helps to confirm the trend and momentum.
  - **RSI**: Identifies overbought and oversold conditions.
  - **Parabolic SAR**: Shows potential reversal points.
  - **EMA**: Provides trend direction, but with short-period settings for 1-minute data.

### Indicator Setup

1. **EMA**: 9-period (short-term trend) and 21-period (medium-term trend) EMAs to define trend direction.
2. **Bollinger Bands**: 20-period with 2 standard deviations, adapted for quick reversals and price channels.
3. **MACD**: Use fast settings, such as 6-period for the short EMA, 13-period for the long EMA, and a 5-period signal line, to react to short-term price movements.
4. **RSI**: 7-period RSI to make it more sensitive to overbought/oversold conditions in a 1-minute context.
5. **Parabolic SAR**: Default settings (with high sensitivity) for trend reversal signals.

### Trading Rules

#### Long Entry Signal

1. **Confirm the Trend**:
   - **EMA Crossover**: The 9 EMA crosses above the 21 EMA, indicating a potential short-term uptrend.
   - **MACD**: MACD histogram or line is positive, showing bullish momentum.

2. **Entry Conditions**:
   - **Bollinger Bands**: Price touches or briefly dips below the lower Bollinger Band.
   - **RSI**: RSI is between 30 and 50, indicating the price is emerging from an oversold state.
   - **Parabolic SAR**: Ensure Parabolic SAR dots are below the price, suggesting an upward trend.

3. **Enter Long Position**: When all conditions are met, enter a long position.

#### Short Entry Signal

1. **Confirm the Trend**:
   - **EMA Crossover**: The 9 EMA crosses below the 21 EMA, suggesting a potential short-term downtrend.
   - **MACD**: MACD histogram or line is negative, showing bearish momentum.

2. **Entry Conditions**:
   - **Bollinger Bands**: Price touches or slightly breaches the upper Bollinger Band.
   - **RSI**: RSI is between 50 and 70, indicating overbought conditions.
   - **Parabolic SAR**: Dots appear above the price, indicating a downward trend.

3. **Enter Short Position**: When all conditions align, enter a short position.

#### Exit Conditions

1. **Long Exit**:
   - **Bollinger Bands**: Price reaches the upper Bollinger Band or moves near it.
   - **RSI**: RSI rises above 70, showing an overbought condition.
   - **Parabolic SAR**: Dots flip to above the price, signaling a potential end of the uptrend.

2. **Short Exit**:
   - **Bollinger Bands**: Price reaches or approaches the lower Bollinger Band.
   - **RSI**: RSI falls below 30, showing an oversold condition.
   - **Parabolic SAR**: Dots flip below the price, indicating a potential end of the downtrend.

#### Stop-Loss and Take-Profit Levels

1. **Stop-Loss**: Place a stop-loss order just outside the recent swing low (for long trades) or swing high (for short trades), typically around 0.2-0.3% for 1-minute data.
2. **Take-Profit**: Set a target based on Bollinger Band levels or RSI extremes, aiming for a quick profit margin of 0.5-1%.

### Example of a 1-Minute Trade

#### Bullish Setup

1. **Trend Confirmation**: 9 EMA crosses above 21 EMA, MACD is positive.
2. **Entry Signal**: Price touches the lower Bollinger Band, RSI is near 40, Parabolic SAR dots appear below price.
3. **Entry**: Enter a long position.
4. **Exit**: Price reaches the upper Bollinger Band, RSI is above 70, or Parabolic SAR dots appear above the price.

#### Bearish Setup

1. **Trend Confirmation**: 9 EMA crosses below 21 EMA, MACD is negative.
2. **Entry Signal**: Price touches the upper Bollinger Band, RSI is near 60, Parabolic SAR dots appear above price.
3. **Entry**: Enter a short position.
4. **Exit**: Price reaches the lower Bollinger Band, RSI falls below 30, or Parabolic SAR dots appear below the price.

### Tips for Real-Time Trading

1. **Use Alerts**: Set up alerts for EMA crossovers, Bollinger Band touches, and Parabolic SAR reversals to help react quickly to signals.
2. **Backtest**: Backtest the strategy on historical 1-minute crypto data to fine-tune parameters based on volatility and crypto type.
3. **Risk Management**: Use a maximum position size per trade and a loss limit per session to manage risk in high-frequency conditions.

This high-frequency strategy can help capitalize on quick, small moves in the crypto market, taking advantage of the volatility seen in minute-by-minute data.

# Long Entry

In [36]:
mask_le1 = (df['ema_short_above_or_below']==1)&(df['flag_grad_ema']==1)&(df['flag_ema_crossing']==1)
mask_le2 = (df['MACD_6_13_5']>0)|(df['MACDh_6_13_5']>0)
mask_le3 = (df['close']<=df['BBL_20_2.0'])
mask_le4 = (df['RSI_7']<50)&(df['RSI_7']>=30)
mask_le5 = (df['PSARl_0.02_0.2'] < df['close'])

In [40]:
df.loc[(mask_le1)&(mask_le2)&(mask_le3)&(mask_le4)&(mask_le5), 'long_entry'] = 1
df.loc[df['long_entry']==1].head()

,date,open,high,low,close,volume,idx_int,gradient_ls,EMA_9,EMA_21,RSI_7,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,MACD_6_13_5,MACDh_6_13_5,MACDs_6_13_5,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2,r_ema_s_m,gradient_ema9,gradient_ema_9,gradient_rsi_7,flag_ema_crossing,flag_grad_ema,flag_grad_rsi,long_entry,short_entry,long_exit,short_exit,ema_short_above_or_below


# Short Entry

In [39]:
mask_se1 = (df['ema_short_above_or_below']==-1)&(df['flag_grad_ema']==-1)&(df['flag_ema_crossing']==1)
mask_se2 = (df['MACD_6_13_5']<0)|(df['MACDh_6_13_5']<0)
mask_se3 = (df['close']>=df['BBU_20_2.0'])
mask_se4 = (df['RSI_7']>50)&(df['RSI_7']<=70)
mask_se5 = (df['PSARs_0.02_0.2'] > df['close'])

In [41]:
df.loc[(mask_se1)&(mask_se2)&(mask_se3)&(mask_se4)&(mask_se5), 'short_entry'] = 1
df.loc[df['short_entry']==1].head()

,date,open,high,low,close,volume,idx_int,gradient_ls,EMA_9,EMA_21,RSI_7,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,MACD_6_13_5,MACDh_6_13_5,MACDs_6_13_5,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2,r_ema_s_m,gradient_ema9,gradient_ema_9,gradient_rsi_7,flag_ema_crossing,flag_grad_ema,flag_grad_rsi,long_entry,short_entry,long_exit,short_exit,ema_short_above_or_below


In [35]:
df.head(30)

,date,open,high,low,close,volume,idx_int,gradient_ls,EMA_9,EMA_21,RSI_7,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,MACD_6_13_5,MACDh_6_13_5,MACDs_6_13_5,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2,r_ema_s_m,gradient_ema9,gradient_ema_9,gradient_rsi_7,flag_ema_crossing,flag_grad_ema,flag_grad_rsi,long_entry,short_entry,long_exit,short_exit,ema_short_above_or_below
0,2024-11-06 12:19:00,2648.29,2648.99,2647.08,2647.79,94.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.02,0.0,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,0
1,2024-11-06 12:20:00,2647.79,2648.21,2641.00,2641.01,124.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2666.640000,0.04,0.0,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,0
2,2024-11-06 12:21:00,2641.00,2645.33,2640.55,2643.85,105.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2665.614400,0.06,0.0,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,0
3,2024-11-06 12:22:00,2643.86,2647.71,2643.86,2647.64,67.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2664.110536,0.06,0.0,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,0
4,2024-11-06 12:23:00,2647.60,2647.60,2644.64,2645.34,398.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2662.696904,0.06,0.0,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,0
5,2024-11-06 12:24:00,2645.34,2647.09,2641.56,2641.91,277.0,5.0,0.173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2661.368090,0.06,0.0,NaN,NaN,0.000000,0.000000,0,0,-1,NaN,NaN,NaN,NaN,0
6,2024-11-06 12:25:00,2642.08,2643.30,2641.34,2642.58,135.0,6.0,0.329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2660.119004,0.06,0.0,NaN,NaN,0.000000,0.000000,0,0,-1,NaN,NaN,NaN,NaN,0
7,2024-11-06 12:26:00,2642.59,2645.17,2642.59,2645.17,346.0,7.0,-0.827,NaN,NaN,49.494970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2658.944864,0.06,0.0,NaN,NaN,0.000000,0.000000,0,0,-1,NaN,NaN,NaN,NaN,0
8,2024-11-06 12:27:00,2644.49,2647.33,2642.96,2644.56,156.0,8.0,-0.770,2644.427778,NaN,46.959563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2657.841172,0.06,0.0,NaN,NaN,0.000000,9.898994,0,0,1,NaN,NaN,NaN,NaN,0
9,2024-11-06 12:28:00,2644.57,2644.86,2639.14,2639.29,143.0,9.0,0.170,2643.400222,NaN,30.969541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2656.803702,0.08,0.0,NaN,NaN,528.885556,14.341410,0,1,1,NaN,NaN,NaN,NaN,0


In [20]:
df.to_clipboard()